<a href="https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2007%20-%20Chains_and_Why_They_Are_Used.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install -q langchain==0.0.208 openai==0.27.8 python-dotenv

In [9]:
import os

# os.environ["OPENAI_API_KEY"]="OPENAI_API_KEY"

from google.colab import userdata
os.environ["OPENAI_API_KEY"]= userdata.get('OPENAI_API_KEY')

# Calling

## __ call __

In [14]:
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI

prompt_template = PromptTemplate(
     input_variables=["word"],
     template="What is a word to replace the following: {word}?",
)


# Set the "OPENAI_API_KEY" environment variable before running following line.
llm =ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

llm_chain = LLMChain(
    llm=llm,
    prompt=prompt_template
)

# Run
result = llm_chain.run(word="artificial")
print(result)


synthetic


## Chain run

In [15]:
input_list = [
    {"word": "artificial"},
    {"word": "intelligence"},
    {"word": "robot"}
]
results = [llm_chain.run(word=d["word"]) for d in input_list]
print(results)

['Synthetic', 'Cleverness', 'android']


## Predict

#### Multiple

In [19]:
prompt_template = "Looking at the context of '{context}'. What is a approapriate word to replace the following: {word}?"

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=prompt_template, input_variables=["word", "context"]))

llm_chain.predict(word="fan", context="object")

'appliance'

In [20]:
llm_chain.predict(word="fan", context="humans")

'supporter'

In [ ]:
# llm_chain.run(word="fan", context="object")

### from string

In [21]:
template = """Looking at the context of '{context}'. What is a approapriate word to replace the following: {word}?"""
llm_chain = LLMChain.from_string(llm=llm, template=template)

In [22]:
llm_chain.predict(word="fan", context="object")

'appliance'

# Parsers

In [23]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
template = """List all possible words as substitute for 'artificial' as comma separated."""

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=template, input_variables=[], output_parser=output_parser))

llm_chain.predict()

'synthetic, man-made, fake, simulated, imitation, faux, counterfeit, ersatz, fabricated, manufactured'

In [24]:
llm_chain.predict_and_parse()

/usr/local/lib/python3.12/dist-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


['synthetic',
 'man-made',
 'fake',
 'simulated',
 'imitation',
 'faux',
 'ersatz',
 'fabricated',
 'counterfeit',
 'pseudo']

# Conversational Chain

In [25]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

template = """List all possible words as substitute for 'artificial' as comma separated.

Current conversation:
{history}

{input}"""

conversation = ConversationChain(
    llm=llm,
    prompt=PromptTemplate(template=template, input_variables=["history", "input"], output_parser=output_parser),
    memory=ConversationBufferMemory())

conversation.predict_and_parse(input="Answer briefly. write the first 3 options.")

['synthetic', 'man-made', 'simulated']

In [26]:
conversation.predict_and_parse(input="And the next 4?")

['imitation', 'fake', 'faux', 'fabricated']

# Debug

In [27]:
conversation = ConversationChain(
    llm=llm,
    prompt=PromptTemplate(template=template, input_variables=["history", "input"], output_parser=output_parser),
    memory=ConversationBufferMemory(),
    verbose=True)

conversation.predict_and_parse(input="Answer briefly. write the first 3 options.")



> Entering new  chain...
Prompt after formatting:
List all possible words as substitute for 'artificial' as comma separated.

Current conversation:


Answer briefly. write the first 3 options.

> Finished chain.


['synthetic', 'man-made', 'simulated']

# Sequential Chain

In [28]:
# from langchain.chains import SimpleSequentialChain
# overall_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

# Custom Chain

In [29]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain

from typing import Dict, List


class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        # Union of the input keys of the two chains.
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}

In [30]:
prompt_1 = PromptTemplate(
    input_variables=["word"],
    template="What is the meaning of the following word '{word}'?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=["word"],
    template="What is a word to replace the following: {word}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("artificial")
print(f"Concatenated output:\n{concat_output}")

Concatenated output:
Artificial means made or produced by human beings rather than occurring naturally. It can also refer to something that is not genuine or real.synthetic
